In [ ]:
#export
import html
import re

from loop.utils import combine

In [ ]:
#export
T_UP   = 'xxup'
T_REP  = 'xxrep'
T_WREP = 'xxwrep'
T_MAJ  = 'xxmaj'
T_BOS  = 'xxbos'
T_EOS  = 'xxeos'
T_FLD  = 'xxfld'
T_UNK  = 'xxunk'
T_PAD  = 'xxpad'
TOKENS = [T_UP, T_REP, T_WREP, T_MAJ, T_BOS, T_EOS, T_FLD, T_UNK, T_PAD]

In [ ]:
def replace_tabs_with_spaces(s): return s.replace('\t', ' ')

In [ ]:
def add_spaces_around(s): return re.sub(r'([/#\n])', r' \1 ', s)

In [ ]:
def trim_useless_spaces(s): return re.sub(' {2,}', ' ', s)

In [ ]:
def replace_repeated_chars(s):
    def _replace(match):
        char, repeats = match.groups()
        return f' {T_REP} {len(repeats) + 1} {char} '
    regex = re.compile(r'(\S)(\1{3,})')
    return regex.sub(_replace, s)

In [ ]:
def replace_repeated_words(s):
    def _replace(match):
        word, repeats = match.groups()
        return f' {T_WREP} {len(repeats.split()) + 1} {word} '
    regex = re.compile(r'(\b\w+\W+)(\1{3,})')
    return regex.sub(_replace, s)

In [ ]:
def fix_wikitext_special_cases(s):
    regex = re.compile(r'  +')
    s = (s.
         replace('#39;',    "'").replace('amp;',    '&').replace('#146;', "'").
         replace('nbsp;',   ' ').replace('#36;',    '$').replace('\\n',  "\n").
         replace('quot;',   "'").replace('<br />', "\n").replace('\\"',   '"').
         replace(' @.@ ',   '.').replace(' @-@ ',   '-').replace(' @,@ ', ',').
         replace('\\',   ' \\ ').replace('<unk>', T_UNK))
    return regex.sub(' ', html.unescape(s))

In [ ]:
def replace_capslock(tokens):
    new = []
    for token in tokens:
        if token.isupper() and len(token) > 1:
            new += [T_UP, token.lower()]
        else:
            new.append(token)
    return new

In [ ]:
def replace_capitalized(tokens):
    new = []
    for token in tokens:
        if token == '':
            continue
        if token[0].isupper() and len(token) > 1 and token[1:].islower():
            new.append(T_MAJ)
        new.append(token.lower())
    return new